## Initial Setup

Let's start by importing some packages.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from tqdm import tqdm
from os import listdir, makedirs
from os.path import join

## Training Parameters

Here we define the hyperparameters for training our model. We recommend keeping the defaults for your first run 🙂

In [ ]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    # GPU
    BATCH_SIZE = 16  # Number of images used in each iteration
    EPOCHS = 3  # Number of passes through entire dataset
    
# Hyperparams for CPU training
else:
    # CPU
    BATCH_SIZE = 4
    EPOCHS = 1

## Data

The dog-breed dataset is already attached to your workspace. It should be available at `/floyd/input/dogbreed`. (if you want to attach your own data, [check out our docs](https://docs.floydhub.com/guides/workspace/#attaching-floydhub-datasets)).

*Note:* We set the `NUM_CLASSES` variable to 10 to keep only the Top 10 dog breeds (so you can train the model on a CPU machine in a reasonable time). Feel free to increase this if you're trying this on a GPU!

In [ ]:
# Keep only the first 10 classes
NUM_CLASSES = 10 

data_dir = '/floyd/input/dogbreed/' # ADD path/to/dataset
labels = pd.read_csv(join(data_dir, 'labels.csv')) # EDIT WITH YOUR LABELS FILE NAME
print("Total number of images in the dataset: {}".format(len(listdir(join(data_dir, 'train'))))) 

print("Top {} labels (sorted by number of samples)".format(NUM_CLASSES))
(labels
 .groupby("breed")
 .count()
 .sort_values("id", ascending=False)
 .head(NUM_CLASSES)
)

### Transfer learning and data augmentation

This is the list of the top labels(dog breeds) sorted by the number of available samples in the dataset. If you notice, we only have ~100 examples of each breed - not much to train a deep learning model from scratch. 

Since we have only a few samples per label, we will
- Set up a Transfer Learning task using the Xception model (pre-trained on ImageNet)
- Use [data augmentation](https://en.wikipedia.org/wiki/Convolutional_neural_network#Artificial_data) to artificially increase the number of training images. We do this by manipulating (scaling, zooming, rotating, etc.) the input images.

Here is an example of what data augmentation does

In [ ]:
from support import data_augmentation_example

input_path = '/floyd/input/dogbreed/train/00a338a92e4e7bf543340dc849230e75.jpg' # An example image from the dataset
count = 9 # Number of samples to show

data_augmentation_example(input_path, count)

## Train - Val Split

We split the dataset into two sets, one for training and one for validation. Splitting the dataset is a common step when the validation and/or test sets are not provided.

In [ ]:
from support import split_train_val

# Make sure to have the same split
SEED = 2018

(train_idx, valid_idx, ytr, yv, labels, selected_breed_list) = split_train_val(labels, NUM_CLASSES, seed=SEED)

from keras.preprocessing import image
from support import show_images

show_images(NUM_CLASSES, labels, data_dir)

## Data Preprocessing

Before feeding the data into the model, we have to preprocess the images.

- The `xception.preprocess_input` method will normalize the value of each pixel to a number between -1 and 1, sample-wise.
- Data augmentation: The `ImageDataGenerator` class will apply the distortions (defined by the parameters) *during* the training process. 

**Note:**: Be careful with data augmentation - the agumentation step is done during the training process and it could [become a bottleneck in the input data pipeline](https://www.tensorflow.org/performance/datasets_performance) if the CPU is not fast enough to provide the preprocessed/augmented data for GPU computation.

In [ ]:
# DATA LOADER
from keras.applications import xception
from support import read_img

INPUT_SIZE = 299 # width/height of image in pixels (as expected by Xception model)

x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, data_dir, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('\nTotal Images shape: {}'.format(x_train.shape))

Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print('Train (images, H, W, C):', Xtr.shape,
      '\nVal (images, H, W, C):', Xv.shape, 
      '\n\nTrain samples (images, labels)', ytr.shape,
      '\nValidation samples (images, labels)', yv.shape)

# Data Loader
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator()

## Model definition

To speed things up, we will apply Transfer Learning by building new layers on top of the pre-trained [Xception](https://arxiv.org/abs/1610.02357) model. 

<img src="https://raw.githubusercontent.com/floydhub/image-classification-template/master/images/xception.png" width="600" height="600" align="center"/>

*Image from the [paper](https://arxiv.org/abs/1610.02357)*

In [ ]:
### MODEL - BOTTLENECK FEATURES - OPTMIZER

from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Model, Input

# Download and create the pre-trained Xception model for transfer learning
base_model = xception.Xception(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have NUM_CLASSES classes
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])
model.summary()

## Train CNN model

If you left the default hyperpameters in the Notebook untouched, your training should take approximately:

- On CPU machine: 15-20 minutes for 1 epoch.
- On GPU machine: 1 minute for 3 epochs (*soo much faster*).

You should get an accuracy of > 96% on the validation set. *Note*: The model will start overfitting after 2 to 3 epochs, since we're already starting from a pre-trained Xception model.

In [ ]:
# TRAINING
hist = model.fit_generator(train_datagen.flow(Xtr, ytr, batch_size=BATCH_SIZE),
                           steps_per_epoch=train_idx.sum() // BATCH_SIZE,
                           epochs=EPOCHS,
                           validation_data=test_datagen.flow(Xv, yv, batch_size=BATCH_SIZE),
                           validation_steps=valid_idx.sum() // BATCH_SIZE)

## Evaluate on validation set

In [ ]:
# EVAL
test_gen = test_datagen.flow(Xv, yv, batch_size=BATCH_SIZE, shuffle=False)
probabilities = model.predict_generator(test_gen)

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
from support import print_confusion_matrix

cnf_matrix = confusion_matrix(np.argmax(probabilities,axis=1), np.argmax(yv, axis=1))
_ = print_confusion_matrix(cnf_matrix, selected_breed_list)

In [ ]:
# Print Precision Recall F1-Score Report
from sklearn.metrics import classification_report

report = classification_report(np.argmax(probabilities,axis=1), np.argmax(yv, axis=1), target_names=selected_breed_list)
print(report)

## It's your turn

Test out the model you just trained. Run the code Cell below and enter a URL with your favorite dog pic in the widget below. Have fun!🎉

*Note*: By default, only the top 10 dog breeds available.

Here are some URLs to test:

- Bernese: https://cdn.pixabay.com/photo/2016/02/06/19/18/dog-1183475_960_720.jpg
- Shih-tzu: https://cdn.pixabay.com/photo/2016/10/08/11/21/shih-tzu-puppy-1723492_960_720.jpg
- Samoyed: https://cdn.pixabay.com/photo/2017/06/14/00/59/samoyed-2400687_960_720.jpg

Can you do better? Play around with the model hyperparameters!

In [ ]:
# Testing on url images
from support import prediction_from_url
from ipywidgets import interact_manual
from ipywidgets import widgets

def get_prediction(URL):
    prediction_from_url(URL, model, selected_breed_list)

interact_manual(get_prediction, URL=widgets.Text(placeholder='Insert URL with a dog pic'));

## Save the result

In [ ]:
# Saving Model Weight
model.save_weights('models/tl_xception_weights.h5')

#### That's all folks - don't forget to shutdown your workspace once you're done 🙂